In [1]:
import pennylane as qml 
import pennylane.numpy as pnp 
import numpy as np 
from qas import qml_ops 
from qas import qml_models 
from qas import mcts
import os 
import json 
from pprint import pprint

dev = qml.device('default.qubit', wires=3)

In [2]:
cwd = os.getcwd() 
print(cwd)

res_file = "20211005-114848.json"
with open(os.path.join(cwd, res_file)) as f:
    res_dict = json.load(f)
print(res_dict.keys())

D:\Research\QAS-Qiskit\QAS
dict_keys(['task', 'pool', 'params', 'k', 'op_list', 'search_reward_list', 'fine_tune_loss'])


In [4]:
pprint(res_dict['op_list'])

[['CRot',
  [0, 1],
  [0.7155452974765524, 0.020417804916402466, -0.713502489836734]],
 ['CRot',
  [1, 2],
  [1.5190876497379346, -0.2546445896530236, -1.5184202417063677]],
 ['CNOT', [0, 2], None],
 ['Rot', [0], [-1.5622544796698883, -0.5413672571695849, 1.5688514279418972]],
 ['Rot', [2], [-1.5706971253874478, 0.4075454388572003, 1.5707010598740383]],
 ['Rot', [1], [-1.7295778023146682, -2.677612052206618, 1.5374237066523149]],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [0], None],
 ['PlaceHolder', [0], None],
 ['PlaceHolder', [0], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None],
 ['PlaceHolder', [0], None],
 ['PlaceHolder', [1], None],
 ['PlaceHolder', [2], None]]


In [34]:
@qml.qnode(dev)
def circ(params, x=None, y=None):
    qml.QubitStateVector(x, wires=[0,1,2])
    qml.Toffoli(wires=[0,1,2])
    """
    qml.CRot(params[0], params[1], params[2], wires=[0,1])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[0,2])
    qml.Rot(params[6], params[7], params[8], wires=[0])
    qml.Rot(params[9], params[10], params[11], wires=[2])
    qml.Rot(params[12], params[13], params[14], wires=[1])
    """
    return qml.state()


def cost(params):
    x_data = qml_models.TOFFOLI_DATA[0]
    y_data = qml_models.TOFFOLI_DATA[1]
    fid_list = []
    for i in range(len(x_data)):
        out=circ(params, x=x_data[i], y=y_data[i])
        print(np.real(np.dot(np.conjugate(out).T, x_data[i])))
        fid_list.append(np.real(np.dot(np.conjugate(out).T, x_data[i])))
    return 1-np.average(fid_list)

dev2 = qml.device('default.qubit', wires=7)

@qml.qnode(dev2)
def circ_swap_test(params, x=None):

    #qml.QubitStateVector(np.kron(x,x), wires = [0,1,2,3,4,5])
    for i in range(6):
        qml.Hadamard(wires=i)

    #qml.Toffoli(wires=[0,1,2])

    
    qml.CRot(params[0], params[1], params[2], wires=[0,1])
    qml.CRot(params[3], params[4], params[5], wires=[1,2])
    qml.CNOT(wires=[0,2])
    qml.Rot(params[6], params[7], params[8], wires=[0])
    qml.Rot(params[9], params[10], params[11], wires=[2])
    qml.Rot(params[12], params[13], params[14], wires=[1])
    
    qml.Toffoli(wires=[3,4,5])

    qml.Hadamard(wires=6)
    qml.CSWAP(wires=[6,5,2])
    qml.CSWAP(wires=[6,4,1])
    qml.CSWAP(wires=[6,3,0])
    qml.Hadamard(wires=6)
    return qml.expval(qml.Hermitian(np.outer(qml_models.ket0, qml_models.ket0), wires=6))

def cost_swap_test(params):
    x_data = qml_models.TOFFOLI_DATA[0]
    fid = 0
    for i in range(len(x_data)):
        out=circ_swap_test(params, x=x_data[i])
        fid = fid + (out-1/2)*2
    return 1-fid/len(x_data)



In [35]:
np.random.seed(0)
var = np.random.randn(15)*0.01

opt = qml.AdamOptimizer(0.5)
num_epochs = 50
for epoch in range(num_epochs):
    var = opt.step(lambda v: cost_swap_test(v), var)
    loss = cost_swap_test(var)
    print("Epoch: {}, Loss: {}".format(epoch+1, loss))

Epoch: 1, Loss: 0.8529455215794299
Epoch: 2, Loss: 0.5748156755431165
Epoch: 3, Loss: 0.18018099169909008
Epoch: 4, Loss: 0.2062975632343037
Epoch: 5, Loss: 0.3459821196998415
Epoch: 6, Loss: 0.27858296120815274
Epoch: 7, Loss: 0.146506045279721
Epoch: 8, Loss: 0.13322466449228088
Epoch: 9, Loss: 0.16433418809931588
Epoch: 10, Loss: 0.1706216032241752
Epoch: 11, Loss: 0.12760166210126145
Epoch: 12, Loss: 0.08427643219220138
Epoch: 13, Loss: 0.0955042523283981
Epoch: 14, Loss: 0.10232747848569523
Epoch: 15, Loss: 0.05888249062559059
Epoch: 16, Loss: 0.030825713185058
Epoch: 17, Loss: 0.05495638979298623
Epoch: 18, Loss: 0.06430730354220748
Epoch: 19, Loss: 0.037266213242957424
Epoch: 20, Loss: 0.02541310728462398
Epoch: 21, Loss: 0.04170720439931774
Epoch: 22, Loss: 0.045367674941694136
Epoch: 23, Loss: 0.03132468816044409
Epoch: 24, Loss: 0.024326750378738105
Epoch: 25, Loss: 0.025683128943880362
Epoch: 26, Loss: 0.02625795035690559
Epoch: 27, Loss: 0.020928935669615956
Epoch: 28, Loss